In [27]:
from preprocessing import *
from swSets import *
from sklearn import naive_bayes, metrics
from sklearn.model_selection import train_test_split, cross_val_score
import scipy
from scipy.sparse import csr_matrix

# <font color ="mediumvioletred">Classifier épisodes dans la bonne série avec Naive Bayes</font>

In [2]:
path = "data" #20 TV-shows, 3108 episodes

In [3]:
corpus = get_corpus(path, texts_as="episode")

In [4]:
df = getTfidfDataFrame(corpus, my_stopwords=stopwords_set)

In [6]:
X = scipy.sparse.csr_matrix(df.values)

In [7]:
d_info, d_name = getDicts(path)

In [28]:
d_name

{1: '10___Smallville',
 2: '122___Malcolm_In_The_Middle',
 3: '12___Doctor_Who',
 4: '132___NCIS',
 5: '15___House',
 6: '16___Desperate_Housewives',
 7: '186___American_Dad',
 8: '2381___Scandal',
 9: '24___Scrubs',
 10: '28___Bones',
 11: '30___Grey_s_Anatomy',
 12: '3103___House_of_Cards_(2013)',
 13: '32___Veronica_Mars',
 14: '366___True_Blood',
 15: '51___How_I_Met_Your_Mother',
 16: '57___CSI__Crime_Scene_Investigation',
 17: '615___The_Good_Wife',
 18: '66___Ugly_Betty',
 19: '71___The_Wire',
 20: '880___Pretty_Little_Liars'}

In [25]:
nb_eps_per_show = []
for i in range(1, len(d_info.keys())+1):
    nb_eps_per_show.append(sum(d_info[i].values()))

In [12]:
#labelisation
Y = []
nb_shows = len(nb_eps_per_show)
somme_cumul = np.cumsum(nb_eps_per_show)
indMin = 0
for i in range(0, nb_shows):
    indMax = somme_cumul[i]
    for j in range(indMin, indMax):
        Y.append(i+1)
    indMin = indMax

In [18]:
#split dataSet and LabelSet into train/test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.20, random_state=42)

In [34]:
#Naives Bayes classifier instanciation
nb_clf = naive_bayes.MultinomialNB()

In [20]:
#train classifier on training data
nb_clf.fit(X_train, Y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [21]:
#predict on test
predictions = nb_clf.predict(X_test)

In [22]:
nb_errors = 0
for i in range(len(Y_test)):
    if Y_test[i] != predictions[i]:
        nb_errors += 1
        
nb_errors

128

In [23]:
#accuracy
nb_clf.score(X_test, Y_test, sample_weight=None)

0.79421221864951763

<font color ="darkorange"><b>Accuracy de 0.79421221864951763</b></font>

# <font color ="mediumvioletred">Cross Validation</font>

### <font color ="forestgreen">5 folds</font>

In [35]:
cv = 5
scores = cross_val_score(nb_clf, X, Y, cv=cv)

In [36]:
scores

array([ 0.74721781,  0.76996805,  0.80483871,  0.81523501,  0.76785714])

In [37]:
np.mean(scores)

0.78102334355956726

<font color ="darkorange"><b>Accuracy de 0.78102334355956726</b></font>

## <font color ="forestgreen">10 folds</font>

In [38]:
cv = 10
scores = cross_val_score(nb_clf, X, Y, cv=cv)

In [39]:
scores

array([ 0.77987421,  0.77672956,  0.78095238,  0.78025478,  0.83757962,
        0.81672026,  0.83116883,  0.82622951,  0.81188119,  0.76490066])

In [40]:
np.mean(scores)

0.80062909964124918

<font color ="darkorange"><b>Accuracy de 0.80062909964124918</b></font>